# Introduction
In this tutorial, we will learn how to use the PuLP library in Python to solve linear programming problems. Linear programming is a method to achieve the best outcome in a mathematical model whose requirements are represented by linear constraints.

In [1]:
#Install pulp
%pip install pulp

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import the library
import random
import matplotlib.pyplot as plt
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, value, LpStatus

## Problem Formulation
Let's consider a simple problem:

Maximize: $$Z = 4x + 3y$$
Subject to:
$$2x + y \leq 20$$
$$x + y \leq 12$$
$$x, y \geq 0$$

In [3]:


# Create a problem variable:
prob = LpProblem("Maximize_the_profit_Z", LpMaximize)

# Create problem variables:
x = LpVariable("x", lowBound=0, upBound=None) # x>=0
y = LpVariable("y", lowBound=0, upBound=None) # y>=0


In linear programming problems, the objective function represents the quantity which needs to be minimized or maximized. It does not have constraints like `<=` or `>=`. On the other hand, constraints are the restrictions or limitations on the variables. They have a certain form based on the problem requirements, often represented with `<=`, `>=`, or `==`.

In [4]:

# The objective function and constraints are added using the += operator to our model.
# Objective function Z
prob += 4*x + 3*y, "Profit"

# Constraints
prob += 2*x + y <= 20
prob += x + y <= 12

<b>Note: The names of variables or constraints must be unique and special characters must not appear, e.g. `=`,`<`,`>`.

In [5]:
# Problem
prob

Maximize_the_profit_Z:
MAXIMIZE
4*x + 3*y + 0
SUBJECT TO
_C1: 2 x + y <= 20

_C2: x + y <= 12

VARIABLES
x Continuous
y Continuous

In [6]:
# Solve the problem
prob.solve()
print("Status:", LpStatus[prob.status])

# Print the optimal production amount of x and y
for v in prob.variables():
    print(v.name, "=", v.varValue)

# Print the optimal profit
print("Total profit is: ", value(prob.objective))

Status: Optimal
x = 8.0
y = 4.0
Total profit is:  44.0


# Second Example: Solving a Knapsack Problem
In this example, we will solve a knapsack problem. We have a set of items, each with a weight and a value, and we want to determine the number of each item to include in a collection so that the total weight is less than or equal to a given limit and the total value is as large as possible.

Maximize:
$$Z = 50x_1 + 100x_2 + 120x_3$$
Subject to:
$$10x_1 + 20x_2 + 30x_3 \leq 50$$
$$x_1, x_2, x_3  \in \{0,1\}$$

In [7]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem(name="Knapsack_Problem", sense=LpMaximize)

# The 3 binary variables that can only take values of 0 or 1
x1 = LpVariable(name="Item1", cat='Binary')
x2 = LpVariable(name="Item2", cat='Binary')
x3 = LpVariable(name="Item3", cat='Binary')

# The objective function is added to 'prob'
prob += lpSum([50*x1, 100*x2, 120*x3]), "Total Value of Items in Knapsack"

# Constraint
prob += lpSum([10*x1, 20*x2, 30*x3]) <= 50, "Total Weight of Items in Knapsack"

In [8]:
# Solve the problem
prob.solve()
print("Status:", LpStatus[prob.status])

# Print the optimal solution
for v in prob.variables():
    print(v.name, "=", v.varValue)

# Print the optimal total value
print("Total value of items in knapsack is: ", value(prob.objective))

Status: Optimal
Item1 = 0.0
Item2 = 1.0
Item3 = 1.0
Total value of items in knapsack is:  220.0


# Third Example: Using Binary Variables as Switches
In this example, we will use a binary variable as a switch to control whether certain constraints are active or not. This is a common technique in linear programming when we want to model conditional constraints.

Maximize: $$Z = b_1 + b_2 + b_3$$
Subject to:
$$x + y + M(1-b_1) \geq 50$$
$$x + 2y + M b_1 \leq 100$$
$$3x + 2y + M(1-b_2) \geq 50$$
$$-x + 5y + M b_3 > 75$$
$$x \geq 0, x \leq 8$$
$$y \geq 0$$
$$b_1, b_2, b_3 \in \{0,1\}$$

In [9]:
prob = LpProblem(name="Switch_Problem", sense=LpMaximize)

# The variables are created
x = LpVariable(name="x", lowBound=0,upBound=8)
y = LpVariable(name="y", lowBound=0)
b1 = LpVariable(name="b1", cat='Binary')
b2 = LpVariable(name="b2", cat='Binary')
b3 = LpVariable(name="b3", cat='Binary')

# The objective function is added to 'prob' first
prob += lpSum([b1,b2,b3]), "Total Value"


M = 1000  # A very large number
eps = 0.00001# A very small number
prob += lpSum([x, y]) + M*(1-b1)>= 50 , "Weight constraint when b1 is 1"
prob += lpSum([x, 2*y]) + M*b1 <= 100 , "Weight constraint when b1 is 0"
prob += lpSum([3*x, 2*y]) + M*(1-b2)>= 50 , "Weight constraint when b2 is 1"
# It is not possible to model sharp inequalities `>` or `<` in solver,
# in order to model them a small epsilon value is added artificially to the non-sharp equations.
prob += lpSum([-x, 5*y]) + M*b3 >= 75+eps , "Weight constraint when b3 is 0"


In [10]:
# Solve the problem
prob.solve()
print("Status:", LpStatus[prob.status])

# Print the optimal solution
for v in prob.variables():
    print(v.name, "=", v.varValue)

# Print the optimal total value
print("Total value is: ", value(prob.objective))

Status: Optimal
b1 = 0.0
b2 = 1.0
b3 = 1.0
x = 8.0
y = 13.0
Total value is:  2.0


# Fourth example: A simplified version of the UTA method

The following is an example code solving the problem of ranking alternatives using the UTA method. Consider a problem with two competing criteria C_1 and C_2.

The following example demonstrates the application of the UTA method for ranking alternatives based on two competing criteria, $C_1$ and $C_2$.  

- Criterion $C_1$ is evaluated using numerical values \($[0, 25, 50, 75, 100]$ (0: least preferred, 100: most preferred)\),

- Criterion $C_2$ is assessed using qualitative ratings \($['Bad', 'Poor', 'Neutral', 'Good', 'Excellent']$ ('Bad': worst, 'Excellent': best)\).

Both criteria are relevant, but their weights can not exceed $0.75$.

In this example, four reference alternatives are randomly assigned values from the available criteria levels. In real-world applications, these values would be determined by a decision maker.

Additionally, the decision maker provides preference information, stating that:
-  $a_1 > a_2$
-  $a_1 > a_3$
-  $a_2 > a_4$

The objective of this setup is not to optimize some objective function but rather to verify whether a system of equations satisfying these preferences has a feasible solution.

Run this example several times and see that for some data sets there is a solution and for some the system of equations is contradictory.

# Weronika Koga 151574

# Zapraszam do wzięcia udziału w podróży moich prób i błędów.

Disclaimer - podróż zaczyna się od 0, a więc nawet jeśli na danym etapie coś wydaje się absolutnie źle, jest wciąż szansa, że zauważyłam to później i poprawiłam. Chciałam jednak pokazać ile pracy w to włożyłam na wypadek gdybym została posądzona przez bezlitosne narzędzie antyplagiatowe o plagiat.

Legenda:
* czesc 0 - pomijalna
* czesc 1 - kierowanie sie wyłącznie maksymalizacją funkcji celu oraz to jak bardzo nie uwzglednia ona preferencji decydenta, dla paru różnych epsilonów
* część 2 - maksymalizacja epsilona

In [ ]:
# czesc 0 - zmienne i dane ktore predzej czy pozniej w swojej nieuwadze i przemęczeniu przypadkiem nadpisze, znajduja sie tu abym mogla je odzyskac. Nic ciekawego.

import pandas as pd
import numpy as np 

# Ustawienia wyświetlania tabeli w jednym wierszu
pd.set_option('display.max_columns', None)  # Wyświetl wszystkie kolumny
pd.set_option('display.expand_frame_repr', False)  # Nie zawijaj wierszy

# Dane wejściowe referencyjne
data_ref = {
    "index": [13, 25, 3, 26, 5, 16, 2, 19, 10, 21],
    "C1": [0.48, 0.34, 1.00, 0.71, 0.62, 0.44, 0.66, 0.35, 0.45, 0.83],
    "C2": [0.97, 1.00, 0.45, 0.25, 0.40, 0.95, 0.55, 0.91, 0.86, 0.25],
    "C3": [0.00, 1.00, 0.57, 0.88, 0.56, 0.00, 0.45, 0.00, 0.00, 0.80],
}
df = pd.DataFrame(data_ref)

# Moglam zrobic skale z krokiem, ale ludzie robili dynamicznie a ze robie sama to ich opinia moze byc jakakolwiek pomoca
all_vals = np.concatenate([df["C1"], df["C2"], df["C3"]])
scale = np.sort(np.unique(np.round(all_vals, 2)))
print("Dynamiczna skala:", scale)

Dynamiczna skala: [0.   0.25 0.34 0.35 0.4  0.44 0.45 0.48 0.55 0.56 0.57 0.62 0.66 0.71
 0.8  0.83 0.86 0.88 0.91 0.95 0.97 1.  ]


In [ ]:
# Lista indeksów moich wariantów oraz epsilony na których będę testować
variant_indices = df["index"].tolist()
eps = [0.00000001, 0.01, 0.05, 1, 0.1, 0.2, 0.333]

# Słownik do przechowywania pozycji wariantów dla każdego epsilon
rankings_table = {variant: [] for variant in variant_indices}

best_objective = -float('inf')  # Najlepsza wartość funkcji celu
best_epsilon = None  # Epsilon dla najlepszej wartości funkcji celu

# Zmienna do przechowywania najlepszych funkcji cząstkowych
best_u_C1_vals = None
best_u_C2_vals = None
best_u_C3_vals = None

for epsilon in eps:
    # Problem optymalizacyjny
    prob = LpProblem("UTA_Method", LpMaximize)

    # Zmienne decyzyjne – cząstkowe funkcje użyteczności
    u_C1 = {val: LpVariable(f"u_C1_{val}", lowBound=0, upBound=1) for val in scale}
    u_C2 = {val: LpVariable(f"u_C2_{val}", lowBound=0, upBound=1) for val in scale}
    u_C3 = {val: LpVariable(f"u_C3_{val}", lowBound=0, upBound=1) for val in scale}

    # Monotoniczność odwrotna (koszt – im wyżej, tym gorzej)
    for i in range(len(scale) - 1):
        prob += u_C1[scale[i]] >= u_C1[scale[i + 1]]
        prob += u_C2[scale[i]] >= u_C2[scale[i + 1]]
        prob += u_C3[scale[i]] >= u_C3[scale[i + 1]]

    prob += u_C1[1] == 0
    prob += u_C2[1] == 0
    prob += u_C3[1] == 0
    prob += u_C1[0] + u_C2[0] + u_C3[0] == 1
    prob += u_C1[0] >= 0.1
    prob += u_C2[0] >= 0.1
    prob += u_C3[0] >= 0.1

    # Funkcja do interpolacji
    def interpolate(val, u_dict):
        lower = max([v for v in scale if v <= val])
        upper = min([v for v in scale if v >= val])
        if lower == upper:
            return u_dict[lower]
        alpha = (val - lower) / (upper - lower)
        return (1 - alpha) * u_dict[lower] + alpha * u_dict[upper]

    # Zmienne dla użyteczności wariantów
    utilities = {}
    for i, row in df.iterrows():
        idx = int(row["index"])
        util = LpVariable(f"U_{idx}", lowBound=0, upBound=1)
        utilities[idx] = util

        uc1 = interpolate(row["C1"], u_C1)
        uc2 = interpolate(row["C2"], u_C2)
        uc3 = interpolate(row["C3"], u_C3)
        prob += util == uc1 + uc2 + uc3

    # Preferencje lokalizacyjne R2 > R1 > R3
    prob += utilities[13] >= utilities[25] + epsilon  # R2 > R3
    prob += utilities[3] >= utilities[26] + epsilon  # R1 > R3
    prob += utilities[5] >= utilities[16] + epsilon  # R2 > R3

    # Preferencje finansowania F1 > F2 > F3
    prob += utilities[19] >= utilities[2] + epsilon  # F1 > F2
    prob += utilities[10] >= utilities[21] + epsilon  # F1 > F3

    # Funkcja celu – maksymalizacja sumy użyteczności
    prob += lpSum([utilities[k] for k in utilities])

    # Rozwiązywanie
    prob.solve()

    # Zapisanie wyników dla tego epsilon
    utility_values = {k: value(v) for k, v in utilities.items()}
    ranking = sorted(utility_values.items(), key=lambda x: x[1], reverse=True)

    # Zapisanie rankingu dla każdego wariantu
    for rank, (idx, val) in enumerate(ranking, 1):
        rankings_table[idx].append(rank)

    # Sprawdzenie, czy to najlepsza wartość funkcji celu
    if value(prob.objective) > best_objective:
        best_objective = value(prob.objective)
        best_epsilon = epsilon

        # Nadpisanie najlepszych funkcji cząstkowych
        best_u_C1_vals = {val: value(u_C1[val]) for val in scale}
        best_u_C2_vals = {val: value(u_C2[val]) for val in scale}
        best_u_C3_vals = {val: value(u_C3[val]) for val in scale}

    print(f"Epsilon = {epsilon}, status = {LpStatus[prob.status]}, objective = {value(prob.objective)}")

# Wyświetlenie najlepszej wartości funkcji celu i epsilon
print(f"\nNajlepsza wartość funkcji celu: {best_objective:.4f} dla epsilon = {best_epsilon}")

# Nadpisanie u_C_vals_old najlepszymi funkcjami cząstkowymi
u_C1_vals_old = best_u_C1_vals
u_C2_vals_old = best_u_C2_vals
u_C3_vals_old = best_u_C3_vals

# Tworzenie tabeli z rankingami
rankings_df = pd.DataFrame(rankings_table, index=[f"Epsilon {e}" for e in eps]).T
rankings_df.index.name = "Wariant"
rankings_df.columns.name = "Epsilon"

# Wyświetlenie tabeli
print("\nTabela rankingów:")
print(rankings_df)

# TODO - rysowanie wykresow

Epsilon = 1e-08, status = Optimal, objective = 8.899999959999999
Epsilon = 0.01, status = Optimal, objective = 8.85
Epsilon = 0.05, status = Optimal, objective = 8.65
Epsilon = 1, status = Infeasible, objective = 3.0
Epsilon = 0.1, status = Optimal, objective = 8.4
Epsilon = 0.2, status = Optimal, objective = 7.8
Epsilon = 0.333, status = Optimal, objective = 7.002

Najlepsza wartość funkcji celu: 8.9000 dla epsilon = 1e-08

Tabela rankingów:
Epsilon  Epsilon 1e-08  Epsilon 0.01  Epsilon 0.05  Epsilon 1  Epsilon 0.1  Epsilon 0.2  Epsilon 0.333
Wariant                                                                                               
13                   1             4             4          5            4            4              4
25                  10            10            10         10           10           10             10
3                    8             8             8          1            5            5              5
26                   9             9  

Co, jak mozna zauwazyc jest bezsensowne bo najlepsze wyniki "podobno" uzyskujemy gdy preferencje finansowania i lokalizacji zupelnie nie sa brane pod uwage

# TODO
- przeanalizowac czy to co wyszlo w epsilon po prostu jest raw najmniejszy koszt i w dupie w ogole F i R oraz usunac ten komentarz bo w dupie to malo akademickie slowo

In [58]:
# Czesc 2 UTA w mam nadzieje sensownej wersji

# Zmienna do przechowywania wyników dla różnych epsilon
rankings_table = {variant: [] for variant in df["index"].tolist()}

best_epsilon_value = -float('inf')  # Najlepsza wartość epsilon
best_epsilon_ranking = None  # Ranking dla najlepszej wartości epsilon

# Problem optymalizacyjny
prob = LpProblem("UTA_Method", LpMaximize)

# Zmienne decyzyjne – cząstkowe funkcje użyteczności
u_C1 = {val: LpVariable(f"u_C1_{val}", lowBound=0, upBound=1) for val in scale}
u_C2 = {val: LpVariable(f"u_C2_{val}", lowBound=0, upBound=1) for val in scale}
u_C3 = {val: LpVariable(f"u_C3_{val}", lowBound=0, upBound=1) for val in scale}

# Zmienna decyzyjna dla epsilon
epsilon = LpVariable("epsilon", lowBound=0)

# Monotoniczność odwrotna (koszt – im wyżej, tym gorzej)
for i in range(len(scale) - 1):
    prob += u_C1[scale[i]] >= u_C1[scale[i + 1]]
    prob += u_C2[scale[i]] >= u_C2[scale[i + 1]]
    prob += u_C3[scale[i]] >= u_C3[scale[i + 1]]

prob += u_C1[1] == 0
prob += u_C2[1] == 0
prob += u_C3[1] == 0
prob += u_C1[0] + u_C2[0] + u_C3[0] == 1
prob += u_C1[0] >= 0.1
prob += u_C2[0] >= 0.1
prob += u_C3[0] >= 0.1

# Funkcja do interpolacji
def interpolate(val, u_dict):
    lower = max([v for v in scale if v <= val])
    upper = min([v for v in scale if v >= val])
    if lower == upper:
        return u_dict[lower]
    alpha = (val - lower) / (upper - lower)
    return (1 - alpha) * u_dict[lower] + alpha * u_dict[upper]

# Zmienne dla użyteczności wariantów
utilities = {}
for i, row in df.iterrows():
    idx = int(row["index"])
    util = LpVariable(f"U_{idx}", lowBound=0, upBound=1)
    utilities[idx] = util

    uc1 = interpolate(row["C1"], u_C1)
    uc2 = interpolate(row["C2"], u_C2)
    uc3 = interpolate(row["C3"], u_C3)
    prob += util == uc1 + uc2 + uc3

# Preferencje lokalizacyjne R2 > R1 > R3
prob += utilities[13] >= utilities[25] + epsilon  # R2 > R3
prob += utilities[3]  >= utilities[26] + epsilon  # R1 > R3
prob += utilities[5]  >= utilities[16] + epsilon  # R2 > R3

# Preferencje finansowania F1 > F2 > F3
prob += utilities[19] >= utilities[2]  + epsilon  # F1 > F2
prob += utilities[10] >= utilities[21] + epsilon  # F1 > F3

# Funkcja celu – maksymalizacja epsilon
prob += epsilon

# Rozwiązywanie
prob.solve()

# Zapisanie wyników
utility_values = {k: value(v) for k, v in utilities.items()}
ranking = sorted(utility_values.items(), key=lambda x: x[1], reverse=True)

# Nadpisanie funkcji cząstkowych z części 1 gdybym chciala wyswietlic czesc 3 dla maksymalizacji epsilona
u_C1_vals_new = {val: value(u_C1[val]) for val in scale}
u_C2_vals_new = {val: value(u_C2[val]) for val in scale}
u_C3_vals_new = {val: value(u_C3[val]) for val in scale}


# Zapisanie rankingu dla każdego wariantu
for rank, (idx, val) in enumerate(ranking, 1):
    rankings_table[idx].append(rank)

# Wyświetlenie wyników
print(f"Najlepsza wartość epsilon: {value(epsilon):.4f}")
print("\nRanking wariantów (od najlepszego):")
for rank, (idx, val) in enumerate(ranking, 1):
    print(f"{rank}. Wariant {idx}: Użyteczność = {val:.4f}")

# Tworzenie tabeli z rankingami
rankings_df = pd.DataFrame(rankings_table, index=["Ranking"]).T
rankings_df.index.name = "Wariant"
rankings_df.columns.name = "Ranking"

# Wyświetlenie tabeli
print("\nTabela rankingów:")
print(rankings_df)

Najlepsza wartość epsilon: 0.3333

Ranking wariantów (od najlepszego):
1. Wariant 5: Użyteczność = 1.0000
2. Wariant 13: Użyteczność = 0.6667
3. Wariant 3: Użyteczność = 0.6667
4. Wariant 16: Użyteczność = 0.6667
5. Wariant 19: Użyteczność = 0.6667
6. Wariant 10: Użyteczność = 0.6667
7. Wariant 25: Użyteczność = 0.3333
8. Wariant 26: Użyteczność = 0.3333
9. Wariant 2: Użyteczność = 0.3333
10. Wariant 21: Użyteczność = 0.3333

Tabela rankingów:
Ranking  Ranking
Wariant         
13             2
25             7
3              3
26             8
5              1
16             4
2              9
19             5
10             6
21            10


In [ ]:
# cześć 3 - UTA z danymi rzeczywistymi
all_data = {
    "index": list(range(1, 28)),
    "C1": [0.60, 0.66, 1.00, 0.48, 0.62, 0.78, 0.40, 0.64, 0.65, 0.45,
           0.61, 0.74, 0.48, 0.69, 0.87, 0.44, 0.68, 0.76, 0.35, 0.64,
           0.83, 0.32, 0.59, 0.73, 0.34, 0.71, 0.80],
    "C2": [0.93, 0.55, 0.45, 0.87, 0.40, 0.27, 0.90, 0.44, 0.30, 0.86,
           0.54, 0.25, 0.97, 0.49, 0.03, 0.95, 0.40, 0.06, 0.91, 0.22,
           0.25, 0.83, 0.24, 0.03, 1.00, 0.25, 0.06],
    "C3": [0.00, 0.45, 0.57, 0.00, 0.56, 0.71, 0.00, 0.54, 0.71, 0.00,
           0.38, 0.80, 0.00, 0.56, 1.00, 0.00, 0.65, 1.00, 0.00, 0.81,
           0.80, 0.00, 0.70, 1.00, 1.00, 0.88, 1.00],
}
df_new = pd.DataFrame(all_data)

# Obliczanie użyteczności nowych wariantów
def interpolated_value(val, scale, u_dict):
    lower = max([v for v in scale if v <= val])
    upper = min([v for v in scale if v >= val])
    if lower == upper:
        return u_dict[lower]
    alpha = (val - lower) / (upper - lower)
    return (1 - alpha) * u_dict[lower] + alpha * u_dict[upper]

# Obliczanie użyteczności nowych wariantów dla u_C_vals_old
results_old = []
for i, row in df_new.iterrows():
    uc1_old = interpolated_value(row["C1"], scale, u_C1_vals_old)
    uc2_old = interpolated_value(row["C2"], scale, u_C2_vals_old)
    uc3_old = interpolated_value(row["C3"], scale, u_C3_vals_old)
    total_old = uc1_old + uc2_old + uc3_old
    results_old.append({"Wariant": row["index"], "Użyteczność_old": round(total_old, 4)})

df_results_old = pd.DataFrame(results_old).sort_values(by="Użyteczność_old", ascending=False).reset_index(drop=True)

# Obliczanie użyteczności nowych wariantów dla u_C_vals_new
results_new = []
for i, row in df_new.iterrows():
    uc1_new = interpolated_value(row["C1"], scale, u_C1_vals_new)
    uc2_new = interpolated_value(row["C2"], scale, u_C2_vals_new)
    uc3_new = interpolated_value(row["C3"], scale, u_C3_vals_new)
    total_new = uc1_new + uc2_new + uc3_new
    results_new.append({"Wariant": row["index"], "Użyteczność_new": round(total_new, 4)})

df_results_new = pd.DataFrame(results_new).sort_values(by="Użyteczność_new", ascending=False).reset_index(drop=True)

# # Porównanie rankingów
# comparison = pd.merge(
#     df_results_old.rename(columns={"Użyteczność_old": "Użyteczność", "index": "Wariant"}).reset_index(),
#     df_results_new.rename(columns={"Użyteczność_new": "Użyteczność", "index": "Wariant"}).reset_index(),
#     on="Wariant",
#     suffixes=("_old", "_new")
# )

# # Dodanie pozycji w rankingu
# comparison["Ranking_old"] = comparison.index + 1
# comparison["Ranking_new"] = comparison.index + 1

# # Wyświetlenie tabeli porównawczej
# print("\nPorównanie rankingów dla u_C_vals_old i u_C_vals_new:")
# print(comparison[["Wariant", "Ranking_old", "Użyteczność_old", "Ranking_new", "Użyteczność_new"]])

# Porównanie rankingów
comparison = pd.merge(
    df_results_old.rename(columns={"Użyteczność_old": "Użyteczność", "index": "Wariant"}).reset_index(),
    df_results_new.rename(columns={"Użyteczność_new": "Użyteczność", "index": "Wariant"}).reset_index(),
    on="Wariant",
    suffixes=("_old", "_new")
)

# Dodanie pozycji w rankingu na podstawie sortowania
comparison["Ranking_old"] = comparison["Użyteczność_old"].rank(ascending=False, method="min").astype(int)
comparison["Ranking_new"] = comparison["Użyteczność_new"].rank(ascending=False, method="min").astype(int)

# Posortowanie tabeli według wariantów dla czytelności
comparison = comparison.sort_values(by="Wariant").reset_index(drop=True)

# Wyświetlenie tabeli porównawczej
print("\nPorównanie rankingów dla u_C_vals_old i u_C_vals_new:")
print(comparison[["Wariant", "Ranking_old", "Ranking_new", "Użyteczność_old", "Użyteczność_new"]])

#TODO rysowanie wykresow




Porównanie rankingów dla u_C_vals_old i u_C_vals_new:
    Wariant  Ranking_old  Ranking_new  Użyteczność_old  Użyteczność_new
0       1.0            9            3           0.8335           0.6667
1       2.0           13           16           0.6670           0.3333
2       3.0           13            3           0.6670           0.6667
3       4.0            1            3           1.0000           0.6667
4       5.0            1            1           1.0000           1.0000
5       6.0           13           16           0.6670           0.3333
6       7.0            1            3           1.0000           0.6667
7       8.0            9            2           0.8335           0.8333
8       9.0           11           15           0.7503           0.4167
9      10.0            1            3           1.0000           0.6667
10     11.0            1            3           1.0000           0.6667
11     12.0           13           16           0.6670           0.3333
12     13

Co to oznacza?

Jest inaczej.

Czy lepiej?

Jest inaczej. 

